In [1194]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import re
import time
import statistics
import json

In [92]:
base_url = 'https://sudact.ru'
folder = 'sverdlovskay_oblast_2021'

In [1385]:
chromedriver = '/home/beaal/Dev/projector2022/projector2022_life_in_debt/chromedriver'
options = webdriver.ChromeOptions()
# options.add_argument('headless')  # для открытия headless-браузера
browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=options)

/tmp/ipykernel_42095/2661307433.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=options)
/tmp/ipykernel_42095/2661307433.py:4: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=options)


In [1386]:
url = 'https://bsr.sudrf.ru/bigs/portal.html#%7B%22rows%22:10,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_doc_result_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222021-10-20T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222021-10-20T23:59:59%5C%22,%5C%22fieldName%5C%22:%5C%22case_user_doc_result_date%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_document_category_article_cat%5C%22,%5C%22operator%5C%22:%5C%22SEW%5C%22,%5C%22query%5C%22:%5C%22%D0%BA%D1%80%D0%B5%D0%B4%D0%B8%D1%82%5C%22,%5C%22fieldName%5C%22:%5C%22case_document_category_article_cat%5C%22%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22simpleSearchFieldsBundle%22:%22default%22,%22noOrpho%22:false,%22groupLimit%22:3,%22woBoost%22:false%7D'
browser.get(url)
time.sleep(10)
# browser.find_element_by_link_text('Гражданское дело 2-5534/2021').click()

In [1366]:
def genederize(fio):
    _sex = fio.split(sep=' ')
    
    if len(_sex) > 3:
        return '-'

    if len(_sex)==3 and '.' not in _sex[:3][-1]:

        _sex = _sex[:3][-1]
        _sex = _sex[-1]

        if _sex=='а':
            return 'Ж'
        else:
            return 'М'
        
    if len(_sex)==2 and '.' in _sex[-1]:
        _sex = _sex[0]
        _sex = _sex[-1]
        if _sex=='а':
            return 'Ж'
        else:
            return '-'
    
    return '-'

def trim(s):
    _s = s.replace('.', ',').split(',')[0].replace(' ', '')
    return _s

In [1365]:
genederize('Мылзенова j. jhh g')

'-'

In [ ]:
for i in range(975,3000):
    row = dict()
    soup_tab_1 = BeautifulSoup(browser.page_source, 'html')

    block_1_dm = ['case_user_doc_number', 'case_common_judicial_uid', 'case_doc_kind', 
                  'case_doc_instance', 'g_case_user_category', 'case_user_doc_court', 
                  'case_doc_subject_rf', 'case_user_doc_result', 'case_common_court_i',
                 'case_links_inet', 'case_user_judge']
    for el in block_1_dm:
        field = soup_tab_1.find("tr", {"data-name": el})
        key = field.contents[0].text.lower()
        value = field.contents[1].text
        if key in row.keys():
            if isinstance(row[key], list):
                row[key].append(value)
            else:
                sub_arr = [row[key]]
                row[key] = sub_arr
                row[key].append(value)
        else:
            row[key] = value

    block2_dm = ['case_common_parts_m2_search']
    a = soup_tab_1.find("tr", {"data-name": 'case_common_parts_m2_search'})
    soup_inner = BeautifulSoup(str(a), 'html')
    b = soup_inner.find_all("tr", {"class": 'one-value'})

    for el in b:
        key = el.contents[0].text.lower()
        if key=='должник':
            key = 'ответчик'
        try:
            value = el.contents[1].text
        except Exception:
            print('no clients info')
            continue
        if key in row.keys():
            if isinstance(row[key], list):
                row[key].append(value)
            else:
                sub_arr = [row[key]]
                row[key] = sub_arr
                row[key].append(value)
        else:
            row[key] = value

    if 'ответчик' in row.keys():
        if isinstance(row['ответчик'], list):
            row['пол ответчика'] = [genederize(el) for el in row['ответчик']]
        else:
            row['пол ответчика'] = genederize(row['ответчик'])

    if 'судья' in row.keys():
        row['пол судьи'] = genederize(row['судья'])
    
    
    try:
        browser.switch_to.default_content()
        time.sleep(2)
        browser.switch_to.frame(0)
        time.sleep(4)
    except Exception:
        browser.switch_to.default_content()
        print('NO IFRAME')
        time.sleep(4)
        _cnt=0
        sl = 5
        while True:
            _cnt +=1
            try:
                browser.find_element_by_xpath(sel_next).click()
            except Exception:
                if _cnt>2 and _cnt <10:
                    print('_cnt>2')
                    sl = 30
                elif _cnt >10:
                    print('_cnt>10')
                    sl = 60
                else:
                    sl = 10
                    print('_cnt second try')
                browser.switch_to.default_content()
                time.sleep(sl)
                browser.switch_to.default_content()
            else:
                break
        continue
        

    src = browser.page_source

    resh = ('решил','Р Е Ш И Л', 'р е ш и л', 'РЕШИЛ')
    reshenie=''
    for el in resh:
        if src.find(el) > -1:
            idx = src.find(el)
            reshenie = src[idx:]
            break


    num_reg = r'(\d[\d\.\,\s]+\d)'   
    after_num = r'(?:\s+\(\D*\))?(?:\sруб\.?\s?\w*\s?\d*\s?(?:коп)?\.?\s?\w*\s?\d*\s?)'

    some_bla = '[^\;\.\d\,]'

    base_num_regex_left = num_reg+after_num+some_bla+r'+'
    base_num_regex_right = r'\w*'+some_bla+'+'+num_reg+after_num

    main_debt_str = r'основ\w+\sдолг'

    regex_main_debt_left = re.compile(base_num_regex_left + main_debt_str, re.M)
    regex_main_debt_right = re.compile(main_debt_str + base_num_regex_right, re.M)

    d1 = regex_main_debt_left.findall(src)
    d2 = regex_main_debt_right.findall(src)

    d = d1+d2

    d = [el.strip() for el in d]


    regex_money = re.compile(num_reg+after_num, re.M)
    res = regex_money.findall(reshenie)
    res = set([el.strip() for el in res])


    if d:
        main_debt = statistics.mode(([int(trim(el)) for el in d]))
        row['основной долг'] = main_debt
    else:
        row['основной долг'] = 0

    if res:
        max_num = max(set([int(trim(el)) for el in res]))
        if main_debt == max_num:
            row['макс выплата'] = sum(set([int(trim(el)) for el in res]))
        else:
            row['макс выплата'] = max_num
    else:
        row['макс выплата'] = 0


    posh_str = '(?:судебн\w+\sрасход\w+|пошлин)'

    regex_posh_debt_left = re.compile(base_num_regex_left+posh_str, re.M)
    regex_posh_debt_right = re.compile(posh_str + base_num_regex_right, re.M)

    posh1 = regex_posh_debt_left.findall(reshenie)
    posh2 = regex_posh_debt_right.findall(reshenie)

    posh = posh1+posh2

    posh = [el.strip() for el in posh]

    if posh:
        if len(posh)<3:
            row['пошлина'] = int(trim(posh[-1]))
        else:
            row['пошлина'] = statistics.mode(([int(trim(el)) for el in posh]))
    else:
        row['пошлина'] = 0


    regex_percent_left = re.compile(base_num_regex_left+'процент', re.M)
    regex_percent_right = re.compile('процент' + base_num_regex_right, re.M)

    proc1_general = regex_percent_left.findall(src)
    proc2_general = regex_percent_right.findall(src)

    proc_general = proc1_general+proc2_general

    proc_general = [el.strip() for el in proc_general]
    proc_general = set(([int(trim(el)) for el in proc_general]))

    proc1_resh = regex_percent_left.findall(reshenie)
    proc2_resh = regex_percent_right.findall(reshenie)

    proc_resh = proc1_resh+proc2_resh

    proc_resh = [el.strip() for el in proc_resh]
    proc_resh = set(([int(trim(el)) for el in proc_resh]))

    fine_str = '(?:штраф|пен|неуст)'

    regex_fine_left = re.compile(base_num_regex_left+fine_str, re.M)
    regex_fine_right = re.compile(fine_str + base_num_regex_right, re.M)

    fine1_general = regex_fine_left.findall(src)
    fine2_general = regex_fine_right.findall(src)

    fine_general = fine1_general + fine2_general

    fine_general = [el.strip() for el in fine_general]
    fine_general = set(([int(trim(el)) for el in fine_general]))

    fine1_resh = regex_fine_left.findall(reshenie)
    fine2_resh = regex_fine_right.findall(reshenie)

    fine_resh = fine1_resh + fine2_resh

    fine_resh = [el.strip() for el in fine_resh]
    fine_resh = set(([int(trim(el)) for el in fine_resh]))



    if proc_resh:
        proc_resh = proc_resh - fine_resh
        row['проценты_решение'] = sum(proc_resh)
        row['проценты_общ'] = 0
        proc_general = 0
    elif proc_general:
        proc_general = proc_general - fine_general
        row['проценты_общ'] = sum(proc_general)
        row['проценты_решение'] = 0
        proc_resh = 0
    else:
        row['проценты'] = 0
        row['проценты_общ'] = 0


    if fine_resh:
        row['пени_решение'] = sum(fine_resh)
        row['пени_общ'] = 0
        fine_general = 0
    elif fine_general:
        row['пени_решение'] = 0
        row['пени_общ'] = sum(fine_general)
        fine_resh = 0
    else:
        row['пени_общ'] = 0
        row['пени_решение'] = 0

    browser.switch_to.default_content()
    time.sleep(4)
    sel_next = '//*[@id="cardContainer"]/div[1]/div/div/span[3]/span'
    cnt=0
    while True:
        cnt +=1
        try:
            browser.find_element_by_xpath(sel_next).click()
        except Exception:
            if cnt>2 and cnt <10:
                print('cnt>2')
                sl = 30
            elif cnt >10:
                print('cnt>10')
                sl = 60
            else:
                sl = 10
                print('second try')
            browser.switch_to.default_content()
            time.sleep(sl)
            browser.switch_to.default_content()
        else:
            break

    print(i)

    with open(f"sudrfdata/json_data/{i}_sudrf.json", "w+",encoding='utf-8') as f:
        s = json.dump(row, f,indent=2, ensure_ascii=False)

    with open(f"sudrfdata/raw_data/{i}_sudrf_raw.txt", "w+") as f:
        header = '|'.join(str(el) for el in row.keys())
        value = '|'.join(str(el) for el in row.values())
        f.write(header)
        f.write(f'\n'+value)

NO IFRAME


/tmp/ipykernel_42095/4155485672.py:71: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_xpath(sel_next).click()


NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME


/tmp/ipykernel_42095/4155485672.py:240: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_xpath(sel_next).click()


982
983
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
992
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
998
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
1007
NO IFRAME
NO IFRAME
1010
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
1015
NO IFRAME
NO IFRAME
NO IFRAME
1019
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
1026
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
1058
NO IFRAME
NO IFRAME
1061
NO IFRAME
NO IFRAME
_cnt second try
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
1071
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
1078
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO IFRAME
NO 

In [ ]:
import pandas as pd

In [1370]:
browser.switch_to.default_content()

In [1320]:
row = dict()
soup_tab_1 = BeautifulSoup(browser.page_source, 'html')

block_1_dm = ['case_user_doc_number', 'case_common_judicial_uid', 'case_doc_kind', 
              'case_doc_instance', 'g_case_user_category', 'case_user_doc_court', 
              'case_doc_subject_rf', 'case_user_doc_result', 'case_common_court_i',
             'case_links_inet', 'case_user_judge']
for el in block_1_dm:
    field = soup_tab_1.find("tr", {"data-name": el})
    key = field.contents[0].text.lower()
    value = field.contents[1].text
    if key in row.keys():
        if isinstance(row[key], list):
            row[key].append(value)
        else:
            sub_arr = [row[key]]
            row[key] = sub_arr
            row[key].append(value)
    else:
        row[key] = value

block2_dm = ['case_common_parts_m2_search']
a = soup_tab_1.find("tr", {"data-name": 'case_common_parts_m2_search'})
soup_inner = BeautifulSoup(str(a), 'html')
b = soup_inner.find_all("tr", {"class": 'one-value'})

for el in b:
    key = el.contents[0].text.lower()
    if key=='должник':
        key = 'ответчик'
    value = el.contents[1].text
    if key in row.keys():
        if isinstance(row[key], list):
            row[key].append(value)
        else:
            sub_arr = [row[key]]
            row[key] = sub_arr
            row[key].append(value)
    else:
        row[key] = value

if 'ответчик' in row.keys():
    if isinstance(row['ответчик'], list):
        row['пол ответчика'] = [genederize(el) for el in row['ответчик']]
    else:
        row['пол ответчика'] = genederize(row['ответчик'])

if 'судья' in row.keys():
    row['пол судьи'] = genederize(row['судья'])


browser.switch_to.default_content()
time.sleep(2)
browser.switch_to.frame(0)
time.sleep(2)

src = browser.page_source

resh = ('решил','Р Е Ш И Л', 'р е ш и л', 'РЕШИЛ')
reshenie=''
for el in resh:
    if src.find(el) > -1:
        idx = src.find(el)
        reshenie = src[idx:]
        break


num_reg = r'(\d[\d\.\,\s]+\d)'   
after_num = r'(?:\s+\(\D*\))?(?:\sруб\.?\s?\w*\s?\d*\s?(?:коп)?\.?\s?\w*\s?\d*\s?)'

some_bla = '[^\;\.\d\,]'

base_num_regex_left = num_reg+after_num+some_bla+r'+'
base_num_regex_right = r'\w*'+some_bla+'+'+num_reg+after_num

main_debt_str = r'основ\w+\sдолг'

regex_main_debt_left = re.compile(base_num_regex_left + main_debt_str, re.M)
regex_main_debt_right = re.compile(main_debt_str + base_num_regex_right, re.M)

d1 = regex_main_debt_left.findall(src)
d2 = regex_main_debt_right.findall(src)

d = d1+d2

d = [el.strip() for el in d]


regex_money = re.compile(num_reg+after_num, re.M)
res = regex_money.findall(reshenie)
res = set([el.strip() for el in res])


if d:
    main_debt = statistics.mode(([int(trim(el)) for el in d]))
    row['основной долг'] = main_debt
else:
    row['основной долг'] = 0

if res:
    max_num = max(set([int(trim(el)) for el in res]))
    if main_debt == max_num:
        row['макс выплата'] = sum(set([int(trim(el)) for el in res]))
    else:
        row['макс выплата'] = max_num
else:
    row['макс выплата'] = 0


posh_str = '(?:судебн\w+\sрасход\w+|пошлин)'

regex_posh_debt_left = re.compile(base_num_regex_left+posh_str, re.M)
regex_posh_debt_right = re.compile(posh_str + base_num_regex_right, re.M)

posh1 = regex_posh_debt_left.findall(reshenie)
posh2 = regex_posh_debt_right.findall(reshenie)

posh = posh1+posh2

posh = [el.strip() for el in posh]

if posh:
    if len(posh)<3:
        row['пошлина'] = int(trim(posh[-1]))
    else:
        row['пошлина'] = statistics.mode(([int(trim(el)) for el in posh]))
else:
    row['пошлина'] = 0


regex_percent_left = re.compile(base_num_regex_left+'процент', re.M)
regex_percent_right = re.compile('процент' + base_num_regex_right, re.M)

proc1_general = regex_percent_left.findall(src)
proc2_general = regex_percent_right.findall(src)

proc_general = proc1_general+proc2_general

proc_general = [el.strip() for el in proc_general]
proc_general = set(([int(trim(el)) for el in proc_general]))

proc1_resh = regex_percent_left.findall(reshenie)
proc2_resh = regex_percent_right.findall(reshenie)

proc_resh = proc1_resh+proc2_resh

proc_resh = [el.strip() for el in proc_resh]
proc_resh = set(([int(trim(el)) for el in proc_resh]))

fine_str = '(?:штраф|пен|неуст)'

regex_fine_left = re.compile(base_num_regex_left+fine_str, re.M)
regex_fine_right = re.compile(fine_str + base_num_regex_right, re.M)

fine1_general = regex_fine_left.findall(src)
fine2_general = regex_fine_right.findall(src)

fine_general = fine1_general + fine2_general

fine_general = [el.strip() for el in fine_general]
fine_general = set(([int(trim(el)) for el in fine_general]))

fine1_resh = regex_fine_left.findall(reshenie)
fine2_resh = regex_fine_right.findall(reshenie)

fine_resh = fine1_resh + fine2_resh

fine_resh = [el.strip() for el in fine_resh]
fine_resh = set(([int(trim(el)) for el in fine_resh]))



if proc_resh:
    proc_resh = proc_resh - fine_resh
    row['проценты_решение'] = sum(proc_resh)
    row['проценты_общ'] = 0
    proc_general = 0
elif proc_general:
    proc_general = proc_general - fine_general
    row['проценты_общ'] = sum(proc_general)
    row['проценты_решение'] = 0
    proc_resh = 0
else:
    row['проценты'] = 0
    row['проценты_общ'] = 0


if fine_resh:
    row['пени_решение'] = sum(fine_resh)
    row['пени_общ'] = 0
    fine_general = 0
elif fine_general:
    row['пени_решение'] = 0
    row['пени_общ'] = sum(fine_general)
    fine_resh = 0
else:
    row['пени_общ'] = 0
    row['пени_решение'] = 0



In [1316]:
browser.switch_to.default_content()

In [ ]:

time.sleep(4)
sel_next = '//*[@id="cardContainer"]/div[1]/div/div/span[3]/span'
cnt=0
while True:
    cnt +=1
    try:
        browser.find_element_by_xpath(sel_next).click()
    except Exception:
        if cnt>2 and cnt <10:
            print('cnt>2')
            sl = 30
        elif cnt >10:
            print('cnt>10')
            sl = 60
        else:
            sl = 10
            print('second try')
        browser.switch_to.default_content()
        time.sleep(sl)
        browser.switch_to.default_content()
    else:
        break

In [1321]:
print(f'основной долг{d}')
print(f'макс выплата {res}')
print(f'пошлина {posh}')
print(f'проценты решение {proc_resh}')
print(f'проценты общ {proc_general}')
print(f'штрафы-пени решение {fine_resh}')
print(f'штрафы-пени общее {fine_general}')
print(str(row))

основной долг['33 614,18']
макс выплата set()
пошлина []
проценты решение 0
проценты общ {6459}
штрафы-пени решение set()
штрафы-пени общее set()
{'номер дела (материала)': '8Г-19031/2021 [88-17757/2021]', 'уникальный идентификатор дела': 'Не заполнено', 'вид судопроизводства': 'Гражданское дело', 'инстанция': 'Кассация', 'категория гражданского и административного дела': 'Иски о взыскании сумм по договору займа, кредитному договору', 'наименование суда': 'Восьмой кассационный суд общей юрисдикции', 'субъект рф': 'Кемеровская область - Кузбасс', 'результат': 'Жалоба / представление ОСТАВЛЕНО БЕЗ УДОВЛЕТВОРЕНИЯ', 'суд (мировой судья) первой инстанции': '04 Егорова И.Н. мировой судья судебного участка Тункинского района Республики Бурятия', 'cсылка на судебное дело на сайте суда': 'Не заполнено', 'судья': 'Соловьев В.Н.- Судья ГР', 'истец': 'ООО НАО " Первое коллекторсое бюро"', 'ответчик': 'Мылзенова Надежда Владимировна', 'представитель': 'Семенова Эльвира Жимбаевна', 'пол ответчика': 

In [1136]:
regex_fine_right

re.compile(r'(?:штраф|пен|неуст)\w*[^\;\.\d\,]{0,35}(\d[\d\.\,\s]+\d)(?:\s+\(\D*\))?(?:\sруб\.?\s?\w*\s?\d*\s?(?:коп)?\.?\s?\w*\s?\d*\s?)',
           re.MULTILINE|re.UNICODE)